In [1]:
import numpy as np
import pandas as pd

In [2]:
# %load ames_feat_transform.py
def trans_features(df):
    
    # drop irrelevant features
    drop_list = pd.read_excel('data/old_features_ames.xlsx', usecols="A,D").dropna()['Feature'].tolist()
    df.drop(columns=drop_list, inplace=True)
    
    # fill missing values
    ofdf = pd.read_excel('data/old_features_ames.xlsx', usecols="A,F")
    ofdf = ofdf[~ofdf['Feature'].isin(drop_list)]
    ofdf.dropna(inplace=True) #only target 'SalePrice' has NaN for 'Missing Handle'
    median_list = ofdf[ofdf['Missing Handle']=='median']['Feature'].tolist()
    mode_list = ofdf[ofdf['Missing Handle']=='mode']['Feature'].tolist()
    
    for med in median_list:
        df[med].fillna(df[med].median(), inplace=True)
    
    for mod in mode_list:
        df[mod].fillna(df[mod].mode()[0], inplace=True)
    
    cusdf = pd.read_excel('data/old_features_ames.xlsx', usecols="A,G")
    cusdf.dropna(inplace=True)
    for index, row in cusdf.iterrows():
        df[row['Feature']].fillna(row['Missing Custom'], inplace=True)
    
    
    # new features
    df['UnfinishedBasement'] = df['BsmtUnfSF'].apply(lambda x: 0 if x==0 else 1)
    df['SecondFloor'] = df['2ndFlrSF'].apply(lambda x: 0 if x==0 else 1)
    df['TotalFullBath'] = df['BsmtFullBath'] + df['FullBath']
    df['TotalHalfBath'] = df['BsmtHalfBath'] + df['HalfBath']
    df['OtherRmsAbvGrd'] = df['TotRmsAbvGrd'] - df['BedroomAbvGr']
    
    # transformations
    log_list = ['LotFrontage','LotArea','TotalBsmtSF','1stFlrSF','GrLivArea']
    for logf in log_list:
        df[logf] = df[logf].apply(lambda x: np.log(x+1))
    
    mssubclass_mapper = {
        20 : 'mssub1',
        30 : 'mssub2',
        40 : 'mssub3',
        45 : 'mssub4',
        50 : 'mssub5',
        60 : 'mssub6',
        70 : 'mssub7',
        75 : 'mssub8',
        80 : 'mssub9',
        85 : 'mssub10',
        90 : 'mssub11',
        120 : 'mssub12',
        150 : 'mssub13',
        160 : 'mssub14',
        180 : 'mssub15',
        190 : 'mssub16'
    }
    
    df['MSSubClass'].replace(mssubclass_mapper, inplace=True)
    #df['MoSold'].replace(mosold_mapper, inplace=True)
    
    df['GarageYrBlt'] = df['GarageYrBlt'] - 1890
    df['YearBuilt'] = df['YearBuilt'] - 1850
    df['YearRemodAdd'] = df['YearRemodAdd'] - 1940
    
    def change_categories(x, cat_dict):
        if x in cat_dict.keys():
            return cat_dict[x]
        elif 'rare_cat_name' in cat_dict.keys():
            return cat_dict['rare_cat_name']
        else:
            return 'others'
    
    
    df['MSZoning'] = df['MSZoning'].apply(lambda x: change_categories(x, {'RL':'RL','FV':'FV'}))
    df['LotShape'] = df['LotShape'].apply(lambda x: change_categories(x, {'Reg':'Reg','rare_cat_name':'Irreg'}))
    df['LandContour'] = df['LandContour'].apply(lambda x: change_categories(x, {'Lvl':'Lvl','Bnk':'Bnk'}))
    df['LotConfig'] = df['LotConfig'].apply(lambda x: change_categories(x, {'Inside':'Inside','FR2':'FR2','Corner':'Corner','rare_cat_name':'CulDFR3'}))
    df['Neighborhood'] = df['Neighborhood'].apply(lambda x: change_categories(x, {'Blmngtn': 'nhood2',  'Blueste': 'nhood3',  'BrDale': 'nhood3',  'BrkSide': 'nhood3',  'ClearCr': 'nhood2',  'CollgCr': 'nhood2',  'Crawfor': 'nhood2',  'Edwards': 'nhood3',  'Gilbert': 'nhood2',  'IDOTRR': 'nhood3',  'MeadowV': 'nhood3',  'Mitchel': 'nhood3',  'NAmes': 'nhood3',  'NPkVill': 'nhood3',  'NWAmes': 'nhood2',  'NoRidge': 'nhood1',  'NridgHt': 'nhood1',  'OldTown': 'nhood3',  'SWISU': 'nhood3',  'Sawyer': 'nhood3',  'SawyerW': 'nhood2',  'Somerst': 'nhood2',  'StoneBr': 'nhood1',  'Timber': 'nhood2',  'Veenker': 'nhood2'}))
    df['Exterior1st'] = df['Exterior1st'].apply(lambda x: change_categories(x, {'AsbShng': 'Ext1_low',  'AsphShn': 'Ext1_low',  'BrkComm': 'Ext1_low',  'BrkFace': 'Ext1_low',  'CBlock': 'Ext1_low',  'CemntBd': 'Ext1_high',  'HdBoard': 'Ext1_low',  'ImStucc': 'Ext1_high',  'MetalSd': 'Ext1_low',  'Plywood': 'Ext1_low',  'Stone': 'Ext1_high',  'Stucco': 'Ext1_low',  'VinylSd': 'Ext1_high',  'Wd Sdng': 'Ext1_low',  'WdShing': 'Ext1_low'}))
    df['Exterior2nd'] = df['Exterior2nd'].apply(lambda x: change_categories(x, {'AsbShng': 'Ext2_low',  'AsphShn': 'Ext2_low',  'Brk Cmn': 'Ext2_low',  'BrkFace': 'Ext2_low',  'CBlock': 'Ext2_low',  'CmentBd': 'Ext2_high',  'HdBoard': 'Ext2_low',  'ImStucc': 'Ext2_high',  'MetalSd': 'Ext2_low',  'Other': 'Ext2_high',  'Plywood': 'Ext2_low',  'Stone': 'Ext2_low',  'Stucco': 'Ext2_low',  'VinylSd': 'Ext2_high',  'Wd Sdng': 'Ext2_low',  'Wd Shng': 'Ext2_low'}))
    df['BldgType'] = df['BldgType'].apply(lambda x: change_categories(x, {'1Fam':'1Fam','TwnhsE':'TwnhsE'}))
    df['HouseStyle'] = df['HouseStyle'].apply(lambda x: change_categories(x, {'1Story':'1Story','1.5Fin':'1.5Fin','2Story':'2Level','2.5Fin':'2Level','rare_cat_name':'other_style'}))
    df['RoofStyle'] = df['RoofStyle'].apply(lambda x: change_categories(x, {'Hip':'Hip','Shed':'Hip','rare_cat_name':'Gable'}))
    df['MasVnrType'] = df['MasVnrType'].apply(lambda x: change_categories(x, {'BrkFace':'Brick','None':'None','BrkCmn':'Brick','Stone':'Stone'}))
    df['ExterQual'] = df['ExterQual'].apply(lambda x: change_categories(x, {'Po':1,'Fa':2,'TA':3,'Gd':4,'Ex':5}))
    #df['ExterCond'] = df['ExterCond'].apply(lambda x: change_categories(x, {'Po':1,'Fa':2,'TA':3,'Gd':4,'Ex':5}))
    df['Foundation'] = df['Foundation'].apply(lambda x: change_categories(x, {'PConc':'PConc','Wood':'PConc','CBlock':'CBlock','Stone':'CBlock','BrkTil':'BrkTil','Slab':'BrkTil'}))
    df['BsmtQual'] = df['BsmtQual'].apply(lambda x: change_categories(x, {'Po':1,'Fa':2,'TA':3,'Gd':4,'Ex':5}))
    df['HeatingQC'] = df['HeatingQC'].apply(lambda x: change_categories(x, {'Po':1,'Fa':2,'TA':3,'Gd':4,'Ex':5}))
    df['Electrical'] = df['Electrical'].apply(lambda x: change_categories(x, {'SBrkr':'SBrkr','rare_cat_name':'FuseBox'}))
    df['KitchenQual'] = df['KitchenQual'].apply(lambda x: change_categories(x, {'Po':1,'Fa':2,'TA':3,'Gd':4,'Ex':5}))
    df['FireplaceQu'] = df['FireplaceQu'].apply(lambda x: change_categories(x, {'Po':1,'Fa':2,'TA':3,'Gd':4,'Ex':5}))
    df['GarageType'] = df['GarageType'].apply(lambda x: change_categories(x, {'Attchd':'Attchd','BuiltIn':'BuiltIn','Detchd':'Detchd','Basment':'Detchd','2Types':'Detchd','NoGarage':'NoGarage','CarPort':'NoGarage'}))
    #df['Fence'] = df['Fence'].apply(lambda x: change_categories(x, {'NoFence':'NoFence','GdPrv':'NoFence','rare_cat_name':'MnPrv'}))
    df['SaleType'] = df['SaleType'].apply(lambda x: change_categories(x, {'New':'New','Con':'New','rare_cat_name':'WD'}))
    df['SaleCondition'] = df['SaleCondition'].apply(lambda x: change_categories(x, {'Normal':'Normal','Partial':'Partial','rare_cat_name':'Abnormal'}))
    
    df['TotalFullBath'] = df['TotalFullBath'].apply(lambda x: 1 if x<1 else 4 if x>4 else x)
    df['TotalHalfBath'] = df['TotalHalfBath'].apply(lambda x: 0 if x==0 else 1)
    df['OtherRmsAbvGrd'] = df['OtherRmsAbvGrd'].apply(lambda x: 1 if x<1 else 8 if x>8 else x)
    
    # drop features no longer relevant
    df.drop(columns=['BsmtUnfSF','2ndFlrSF','BsmtHalfBath'], inplace=True)
    
    return df

In [3]:
normalize_list = pd.read_excel('data/old_features_ames.xlsx', usecols="A,K").dropna()['Feature'].tolist()
normalize_list.extend(['TotalFullBath','TotalHalfBath','OtherRmsAbvGrd'])

In [4]:
df = pd.read_csv('data/train.csv')

In [5]:
df = trans_features(df)

In [6]:
df.shape

(1460, 52)

In [7]:
df.isnull().any().sum()

0

In [8]:
from sklearn.preprocessing import StandardScaler

In [9]:
scaler = StandardScaler()
df2 = df.copy()
df2[normalize_list] = scaler.fit_transform(df2[normalize_list])
cat_list = df2.select_dtypes(exclude=np.number).columns.tolist()
df2_cat = pd.get_dummies(df2[cat_list])
df2.drop(columns=cat_list, inplace=True)
df2 = pd.concat([df2,df2_cat], axis=1)

/anaconda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/anaconda/lib/python3.6/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [10]:
df2.shape

(1460, 113)

In [11]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

from sklearn.model_selection import cross_val_score

In [15]:
from sklearn.metrics import mean_squared_error

In [36]:
lr = LinearRegression()

In [37]:
cross_val_score(lr, df2.drop(columns=['SalePrice']), df2['SalePrice'].apply(lambda x: np.log(x+1)), cv=5)

array([ 8.96962276e-01,  8.84185697e-01, -5.21307189e+20, -4.61181394e+20,
        8.23568959e-01])

In [40]:
rf = RandomForestRegressor(n_estimators=60)
cross_val_score(rf, df2.drop(columns=['SalePrice']), df2['SalePrice'].apply(lambda x: np.log(x+1)), cv=5)

array([0.86482672, 0.86319123, 0.87328643, 0.8857346 , 0.84480807])

In [41]:
xtrain, xtest, ytrain, ytest = train_test_split(df2.drop(columns=['SalePrice']), df2['SalePrice'].apply(lambda x: np.log(x+1)), test_size=0.3, random_state=42)

In [42]:
rf2 = RandomForestRegressor(n_estimators=60)
rf2.fit(xtrain, ytrain)
mean_squared_error(ytest, rf2.predict(xtest))

0.01964198060574307

In [24]:
from sklearn.preprocessing import Normalizer

In [25]:
scaler = Normalizer()
df3 = df.copy()
df3[normalize_list] = scaler.fit_transform(df3[normalize_list])
cat_list = df3.select_dtypes(exclude=np.number).columns.tolist()
df3_cat = pd.get_dummies(df3[cat_list])
df3.drop(columns=cat_list, inplace=True)
df3 = pd.concat([df3,df3_cat], axis=1)

In [23]:
from keras import models
from keras import layers

/anaconda/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [26]:
from sklearn.model_selection import train_test_split

In [30]:
features_train, features_test, target_train, target_test = train_test_split(df3.drop(columns=['SalePrice']), df3['SalePrice'].apply(lambda x: np.log(x+1)), test_size=0.3, random_state=42)

In [43]:
amesnet = models.Sequential()
amesnet.add(layers.Dense(units=64, activation='relu', input_shape=(features_train.shape[1],)))
amesnet.add(layers.Dense(units=64, activation='relu'))
#amesnet.add(layers.Dense(units=32, activation='relu'))
amesnet.add(layers.Dense(units=1))
amesnet.compile(loss='mse',
                optimizer='RMSprop',
                metrics=['mse'])
history = amesnet.fit(features_train,
                     target_train,
                     epochs=3,
                     verbose=1,
                     batch_size=100,
                     validation_data=(features_test, target_test))

Train on 1022 samples, validate on 438 samples
Epoch 1/3
1022/1022 [==============================] - 0s 125us/step - loss: 112.5875 - mean_squared_error: 112.5875 - val_loss: 60.0066 - val_mean_squared_error: 60.0066
Epoch 2/3
1022/1022 [==============================] - 0s 35us/step - loss: 29.9241 - mean_squared_error: 29.9241 - val_loss: 3.9396 - val_mean_squared_error: 3.9396
Epoch 3/3
1022/1022 [==============================] - 0s 37us/step - loss: 1.5002 - mean_squared_error: 1.5002 - val_loss: 0.5461 - val_mean_squared_error: 0.5461


In [32]:
111 ** (1/2)

10.535653752852738

In [35]:
df['SalePrice'].apply(lambda x: np.log(x+1)).max()

13.534474352733596